In [5]:
%load_ext autoreload
%autoreload 2
import logging_setup
print(logging_setup.__file__)
import sys, os
import pandas as pd
import time
import logging
from pythonjsonlogger import jsonlogger
from flumine import FlumineSimulation, clients
from collections import OrderedDict, deque, defaultdict
from flumine import BaseStrategy
from flumine.order.trade import Trade
from flumine.order.order import OrderStatus
from flumine.order.ordertype import LimitOrder
from flumine.utils import get_price
from logging_setup import build_logger
import random
import time
from datetime import timedelta

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/Users/hcoussens/git/betfair_profitbox/research/research_notebooks/logging_setup.py


In [6]:
class FlumineStrat(BaseStrategy):
    """
    Example strateg
    """
    def __init__(self, enter_threshold, exit_threshold, order_hold, price_add, log_root, log_level, *a, **k):
        self.log = build_logger(log_root,log_level)  # logs/trades.log, rotated nightly
        super().__init__(name="risk_backfave",*a, **k)
        self.hist = defaultdict(lambda: deque(maxlen=400))  # per runner
        self.enter_threshold = enter_threshold
        self.exit_threshold = exit_threshold
        self.order_hold = order_hold
        self.price_add = price_add
        self.startdt = None
        self._last = {}    # order_id -> last size_matched
        self.rows = []     # collected fills: [market_id, selection_id, time, size, price, side, order_id]
        self.pnl = 0.0     


    def add_market(self, market):
        self.log.info("ADD", market.market_id, market.event_name, market.event_type_id)
    
    def check_market_book(self, market, market_book):
        if market_book.status == "OPEN" and market_book.inplay:
            return True
    
    def _price_now(self, r):
        return r.last_price_traded
    
    def _price_n_secs_ago(self, key, now_dt, n=5):
        cutoff = now_dt - timedelta(seconds=n)
        dq = self.hist.get(key)
        if not dq: return None
        for t,p in reversed(dq):
            if t <= cutoff:
                return p
        return None

    def matched_summary(self, r, market):
        back_total = lay_total = 0.0
        back_weighted = lay_weighted = 0.0
    
        for o in market.blotter:
            if o.selection_id != r.selection_id:
                continue
            m = float(getattr(o, "size_matched", 0) or 0)
            if m <= 0:
                continue
            p = float(getattr(o, "average_price_matched", 0) or 0)
            side = str(getattr(o, "side", "")).upper()
            if "BACK" in side:
                back_total += m
                back_weighted += m * p
            elif "LAY" in side:
                lay_total += m
                lay_weighted += m * p
    
        avg_back = back_weighted / back_total if back_total else 0.0
        avg_lay = lay_weighted / lay_total if lay_total else 0.0
        return back_total, avg_back, lay_total, avg_lay

    def best_prices_for_runner(self, r):
        ex = getattr(r, "ex", None)
        atb = getattr(ex, "available_to_back", []) or []
        atl = getattr(ex, "available_to_lay", []) or []
    
        def _extract(item):
            # handle dict or PriceSize
            if isinstance(item, dict):
                return item.get("price"), item.get("size")
            return getattr(item, "price", None), getattr(item, "size", None)
            
        bb_price, bb_size = _extract(atb[0]) if atb else (None, None)
        bl_price, bl_size = _extract(atl[0]) if atl else (None, None)
        return bb_price, bb_size, bl_price, bl_size

    
    def process_market_book(self, market, market_book):
        try:
            if not self.startdt: self.startdt = market_book.publish_time
            elapsed = market_book.publish_time.timestamp() - self.startdt.timestamp()
            # self.log.debug(f"Process_market_book: {market.event_name} {market.market_id}, time elapsed {elapsed},  publishtime:{market_book.publish_time}")
    
            if elapsed > 1:
                for r in market_book.runners:
                    now_dt = market_book.publish_time
                    context = f"->{now_dt.strftime("%H:%M:%S.") + f"{int(now_dt.microsecond/100000)}"}|{market.market_id}|{r.selection_id}|"
                    back_total, avg_back, lay_total, avg_lay = self.matched_summary(r,market)
                    runner_context = self.get_runner_context(market.market_id, r.selection_id, r.handicap)

                    key = (market.market_id, r.selection_id)
                    p = self._price_now(r)
                    if not p : return
                    self.log.debug(f"Market tick for {context}, price {p}")
                    
                    if p and p < self.enter_threshold:
                        if runner_context.live_trade_count == 0:
                            self.log.info(f"Trigger back trade: {context} placing order, price {p}")
                            back = round(get_price(r.ex.available_to_lay, 0) + self.price_add,2)
                            trade = Trade(market_book.market_id, r.selection_id, r.handicap, self, notes={"entry_px": back})
                            order = trade.create_order(side="BACK", 
                                                       order_type=LimitOrder(back, self.context["stake"])
                                                       )
                            try:
                                market.place_order(order)
                            except Exception as e:
                                self.log.warning(str(e)) 
                            self.log.info({"ORDER PLACED": context, "price":back})
    
                    if p > self.exit_threshold and back_total:
                        bestb, _ , bestl ,_ = self.best_prices_for_runner(r)
                        loss_on_loss_covered_prc = round(lay_total / back_total,2)
                        cover_ratio = 0.3
                        if loss_on_loss_covered_prc < cover_ratio:
                            if runner_context.live_trade_count == 0:
                                self.log.warning(f"Seeing reason to hedge: {context}, ltp {p} \
                                \n back_total:{back_total}, avg_back:{avg_back}, lay_total:{lay_total}, avg_lay:{avg_lay}.\
                        \n Loss Cover Ratio:{lay_total}/{back_total} = {loss_on_loss_covered_prc} : Hedging as ratio < {cover_ratio} \
                        \n Bestback : {bestb} , bestlay : {bestl}")
                                self.hedge_selection(r, market, market_book, p, context, size=2, price=bestl)
                                
        except Exception as e:
            self.log.warning(f"Failed to process market book : {str(e)}")
            
    def hedge_selection(self,r, market, market_book, p, context, size, price):
        try:
            self.log.warning(f"LAY order pre send : {r.selection_id}, hedge size {size}@ hedge price {price}")
            trade = Trade(market_book.market_id, r.selection_id, r.handicap, self)
            order = trade.create_order("LAY", order_type=LimitOrder(price, size))
            market.place_order(order)
            self.log.warning(f"LAY order placed : {context} " )
        except Exception as e:
            self.log.warning(f"Failed send LAY order  : {str(e)}")
            

    def process_orders(self, market, orders):
        for order in orders:
            if order.status == OrderStatus.EXECUTABLE:
                if order.elapsed_seconds and order.elapsed_seconds > self.order_hold:
                     market.cancel_order(order)

    def process_closed_market(self, market, market_book):
        self.pnl = 0.0
        self.log.info(f"Processing closed market: {market.event_name}, {market.market_id}")
        for order in market.blotter:
            self.pnl += order.profit
            self.log.warning(f"Order PNL {order.profit}, av size matched: {order.size_matched} av price matched: {order.average_price_matched}, date_time_created: {order.date_time_created}")
        self.log.warning(f"Total pnl for market:{market.event_name}, {market.market_id}, : PNL :: {self.pnl}")
    

In [7]:
def run_once(market, max_order_exposure, max_selection_exposure, stake, enter_threshold, exit_threshold, order_hold, price_add):
    client = clients.SimulatedClient()
    framework = FlumineSimulation(client=client)

    lists =[]
    lists.append(market)
    
    strategy = FlumineStrat(   
        market_filter={"markets": lists},
        max_order_exposure=max_order_exposure,
        max_selection_exposure=max_selection_exposure,
        context={"stake": stake},
        enter_threshold=enter_threshold,
        exit_threshold=exit_threshold,
        order_hold=order_hold,
        price_add=price_add,
        log_root="./logs/backtest/",
        log_level="W")

    framework.add_strategy(strategy)
    framework.run()
    pnl = getattr(strategy, "pnl")
    del framework, client, strategy
    return pnl

In [8]:
all_bought_data = pd.read_csv("bought_data_catalogue.csv")["path"].tolist()
!pwd
bought_single = "../hist_data/ADVANCED/2025/Feb/12/33524813/1.232245129"
mine_single = "../hist_data/self_recorded/2025-10-09/4/1.248809424.jsonl"
use = "../hist_data/ADVANCED/2024/Aug/18/33500547/1.231887491"


pnl = run_once(use,
         max_order_exposure=30,
         max_selection_exposure=30,
         stake=2,
         enter_threshold=1.2,
         exit_threshold=5.9, 
         order_hold=17,
         price_add=0.01)

print(pnl)



/Users/hcoussens/git/betfair_profitbox/research/research_notebooks


FileNotFoundError: [Errno 2] No such file or directory: '../hist_data/ADVANCED/2024/Aug/18/33500547/1.231887491'